# LLM Catan Arena - Interactive Analysis

This notebook provides comprehensive analysis of LLM Catan games with interactive visualizations and detailed breakdowns.

## 1. Setup & Data Loading

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

# Import our analysis modules
from src.analysis import (
    load_all_games,
    calculate_win_rates,
    calculate_costs,
    calculate_game_stats,
    head_to_head_matrix,
    export_summary_report,
)
from src.visualizations import (
    plot_win_rates,
    plot_cost_efficiency,
    plot_cost_per_win,
    plot_game_length_distribution,
    plot_head_to_head,
    plot_decision_speed,
    plot_token_usage,
    generate_all_visualizations,
)
from src.highlight_finder import (
    find_interesting_moments,
    get_game_details,
    analyze_model_patterns,
)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports successful")

In [ ]:
# Load all games
DATA_DIR = "../data/games"
games = load_all_games(DATA_DIR)

print(f"📊 Loaded {len(games)} games")

# Display sample game structure
if games:
    print("\n📋 Sample game metadata:")
    sample = games[0]
    print(f"  Session ID: {sample.get('session_id')}")
    print(f"  Players: {sample.get('players')}")
    print(f"  Winner: {sample.get('winner')}")
    print(f"  Final Scores: {sample.get('final_scores')}")
    print(f"  Total Moves: {len(sample.get('moves', []))}")

## 2. Overall Summary Statistics

In [ ]:
# Calculate overall stats
stats = calculate_game_stats(games)

display(Markdown(f"""
### Overall Statistics

- **Total Games**: {stats['total_games']}
- **Average Game Length**: {stats['avg_turns']:.1f} turns (median: {stats['median_turns']:.0f})
- **Game Length Range**: {stats['min_turns']:.0f} - {stats['max_turns']:.0f} turns
- **Average Duration**: {stats['avg_duration_minutes']:.1f} minutes (median: {stats['median_duration_minutes']:.1f})
- **Average Winning Score**: {stats['avg_winning_score']:.1f} VP
- **Average Victory Margin**: {stats['avg_victory_margin']:.1f} VP (median: {stats['median_victory_margin']:.1f})
"""))

## 3. Win Rate Analysis

In [ ]:
# Calculate win rates
win_rates = calculate_win_rates(games)

display(Markdown("### Win Rates by Model"))
display(win_rates.style.format({
    'win_rate': '{:.1%}',
}).background_gradient(subset=['win_rate'], cmap='RdYlGn'))

In [ ]:
# Visualize win rates
plot_win_rates(games, "../output/charts/win_rates_notebook.png")
plt.show()

## 4. Cost Analysis

In [ ]:
# Calculate costs
costs = calculate_costs(games)

display(Markdown("### Cost Statistics"))
display(costs.style.format({
    'total_cost': '${:.4f}',
    'avg_cost_per_game': '${:.4f}',
    'cost_per_win': '${:.4f}',
}).background_gradient(subset=['avg_cost_per_game'], cmap='RdYlGn_r'))

In [ ]:
# Cost efficiency scatter plot
plot_cost_efficiency(games, "../output/charts/cost_efficiency_notebook.png")
plt.show()

In [ ]:
# Cost per win
plot_cost_per_win(games, "../output/charts/cost_per_win_notebook.png")
plt.show()

## 5. Head-to-Head Analysis

In [ ]:
# Head-to-head matrix
h2h = head_to_head_matrix(games)

display(Markdown("### Head-to-Head Win Rates"))
display(Markdown("*Win rate when ROW model plays against COLUMN model*"))
display(h2h.style.format('{:.1%}').background_gradient(cmap='RdYlGn', vmin=0, vmax=1))

In [ ]:
# Visualize head-to-head
plot_head_to_head(games, "../output/charts/head_to_head_notebook.png")
plt.show()

## 6. Game Dynamics

In [ ]:
# Game length distribution
plot_game_length_distribution(games, "../output/charts/game_length_notebook.png")
plt.show()

In [ ]:
# Decision speed comparison
plot_decision_speed(games, "../output/charts/decision_speed_notebook.png")
plt.show()

In [ ]:
# Token usage patterns
plot_token_usage(games, "../output/charts/token_usage_notebook.png")
plt.show()

## 7. Interesting Moments Explorer

In [ ]:
# Find interesting moments
highlights = find_interesting_moments(games)

display(Markdown("### Interesting Games by Category"))

for category, game_ids in highlights.items():
    if game_ids:
        category_name = category.replace('_', ' ').title()
        print(f"\n{category_name}: {len(game_ids)} games")
        for game_id in game_ids[:3]:  # Show first 3
            details = get_game_details(games, game_id)
            if details:
                print(f"  • {game_id}")
                print(f"    Winner: {details['winner']} ({details['winner_score']} VP, margin: {details['victory_margin']})")

## 8. Deep Dive: Sample Game Analysis

In [ ]:
# Pick an interesting game to analyze
if highlights["close_finishes"]:
    interesting_game_id = highlights["close_finishes"][0]
else:
    interesting_game_id = games[0].get('session_id')

game_details = get_game_details(games, interesting_game_id)

display(Markdown(f"""
### Game Analysis: {interesting_game_id}

**Winner**: {game_details['winner']} ({game_details['winner_score']} VP)  
**Victory Margin**: {game_details['victory_margin']} VP  
**Game Length**: {game_details['total_turns']} turns  
**Duration**: {game_details['duration_minutes']:.1f} minutes  
**Total Cost**: ${game_details['total_cost']:.4f}

#### Final Scores
"""))

scores_df = pd.DataFrame([
    {'Player': player, 'Score': score, 'Winner': '👑' if player.endswith(game_details['winner'].split(':')[-1]) else ''}
    for player, score in sorted(game_details['final_scores'].items(), key=lambda x: x[1], reverse=True)
])
display(scores_df)

In [ ]:
# Analyze move-by-move for this game
game = next((g for g in games if g.get('session_id') == interesting_game_id), None)

if game:
    moves = game.get('moves', [])
    
    # Sample some moves
    display(Markdown("#### Sample Moves"))
    sample_indices = [0, len(moves)//4, len(moves)//2, 3*len(moves)//4, -1]
    
    sample_moves = []
    for idx in sample_indices:
        if 0 <= idx < len(moves) or idx == -1:
            move = moves[idx]
            sample_moves.append({
                'Turn': move.get('turn_number', 0),
                'Player': move.get('player', ''),
                'Action': move.get('move_data', {}).get('action', 'Unknown'),
                'Cost': f"${move.get('move_data', {}).get('cost', 0):.5f}",
                'Tokens': move.get('move_data', {}).get('tokens', 0),
            })
    
    display(pd.DataFrame(sample_moves))

## 9. Model Pattern Analysis

In [ ]:
# Analyze model-specific patterns
model_patterns = analyze_model_patterns(games)

display(Markdown("### Model-Specific Patterns"))

patterns_df = pd.DataFrame([
    {
        'Model': model,
        'Total Moves': patterns['total_moves'],
        'Avg Cost/Move': f"${patterns['avg_cost_per_move']:.5f}",
        'Avg Response Time': f"{patterns['avg_response_time']:.2f}s",
        'Most Common Action': patterns['most_common_action'],
    }
    for model, patterns in model_patterns.items()
])

display(patterns_df)

## 10. Export for Blog

In [ ]:
# Generate all charts
print("📈 Generating all visualizations...")
generate_all_visualizations(games, "../output/charts")
print("✅ Visualizations saved to output/charts/")

In [ ]:
# Generate summary report
print("📝 Generating summary report...")
export_summary_report(games, "../output/summary_report.md")
print("✅ Report saved to output/summary_report.md")

In [ ]:
# Generate highlights report
from src.highlight_finder import generate_highlights_report

print("🎯 Generating highlights report...")
generate_highlights_report(games, "../output/highlights_report.md")
print("✅ Highlights saved to output/highlights_report.md")

In [ ]:
# Generate blog draft
from scripts.generate_blog_draft import generate_blog_draft

print("✍️  Generating blog draft...")
generate_blog_draft(games, "../output/blog_draft.md")
print("✅ Blog draft saved to output/blog_draft.md")

## 11. Custom Analysis

In [ ]:
# Add your own custom analysis here
# For example, filter games by specific models, date ranges, etc.

# Example: Analyze only games with specific models
# filtered_games = [g for g in games if 'gpt4' in g.get('players', [])[0].lower()]
# filtered_win_rates = calculate_win_rates(filtered_games)
# display(filtered_win_rates)